In [ ]:
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image

json_path = input().strip()
with open(json_path, "r", encoding="utf-8") as f:
    plant_data = json.load(f)

class_keys = sorted(list(plant_data.keys()))

def load_and_preprocess_image(img_path, target_size=(224,224)):
    img = image.load_img(img_path, target_size=target_size)
    arr = image.img_to_array(img) / 255.0
    return np.expand_dims(arr, axis=0)

def predict_topk(img_path, top_k=5):
    x = load_and_preprocess_image(img_path, (img_size, img_size))
    preds = medinet_xg.predict(x, verbose=0)[0]
    idx = preds.argsort()[-top_k:][::-1]
    return [(class_keys[i], float(preds[i]) * 100.0) for i in idx]

def generate_explanation(plant_key):
    info = plant_data.get(plant_key, {})
    if not info:
        return "No information available."
    return (
        f"{info.get('Scientific Name','Unknown')} ({plant_key}) is a {info.get('Life Span','')} {info.get('Plant Habit','')} "
        f"in the {info.get('Botanical Family','')} family. It grows in {info.get('Native Habitat','')} and originates from "
        f"{info.get('Geographical Origin','')}. Traditional uses: {info.get('Traditional Uses','')}. Active compounds: "
        f"{info.get('Primary Active Compounds','')}. Climate: {info.get('Climate Requirements','')}. Soil: "
        f"{info.get('Preferred Soil Type','')}. Safety: {info.get('Pregnancy & Lactation Safety','')}. Leaf: "
        f"{info.get('Leaf Morphology','')}."
    )

def answer_query(plant_key, query):
    info = plant_data.get(plant_key, {})
    if not info:
        return "No plant information found."
    q = query.lower()
    keywords = {
        "scientific": "Scientific Name",
        "botanical": "Botanical Family",
        "genus": "Genus & Species",
        "family": "Botanical Family",
        "leaf": "Leaf Morphology",
        "life": "Life Span",
        "habitat": "Native Habitat",
        "origin": "Geographical Origin",
        "uses": "Traditional Uses",
        "medicinal": "Therapeutic Properties",
        "dosage": "Standard Dosage",
        "toxic": "Toxicity Levels",
        "safety": "Pregnancy & Lactation Safety",
        "interaction": "Drug-Herb Interactions",
        "soil": "Preferred Soil Type",
        "climate": "Climate Requirements"
    }
    for k, json_key in keywords.items():
        if k in q:
            return info.get(json_key, "No information available for this query.")
    return "Outside the current knowledge base."

img_path = input().strip()
query = input().strip()
topk = predict_topk(img_path, top_k=5)
pred_class = topk[0][0]
confidence = topk[0][1]

print(pred_class)
print(confidence)
print(topk)
print(generate_explanation(pred_class))
print(answer_query(pred_class, query))
